This code implements the algorythim described in the article [A constructive algorithm for the Cartan decomposition of SU(2N)](https://arxiv.org/abs/quant-ph/0505128). In the comments we will reference equation from the article and indicate witch step from the "step by step sumary" each part of the code is implementing.

In this block we import packages and define functions that will be used in the decomposition

In [ ]:
#Import packages

import numpy as np
import math
from numpy import log as ln
import scipy as sc
from scipy.linalg import expm, logm
import functools as ft
import pandas as pd
from IPython.display import display
import random
import itertools

# Define Pauli matrices

X=np.array([[0, 1], [1, 0]])
Y=np.array([[0, -1j], [1j, 0]])
Z=np.array([[1, 0], [0, -1]])
I=np.array([[1, 0], [0, 1]])


# Define a dictionary for symbol to matrix conversion
symbol_to_matrix = {
    'x': X,
    'y': Y,
    'z': Z,
    'i': I,
}


# Define KG basis for su(4)

listOfm_2 = []
listOfm_2.append((1j/2)*np.kron(X,X))
listOfm_2.append((1j/2)*np.kron(Y,Y))
listOfm_2.append((1j/2)*np.kron(Z,Z))
listOfm_2.append((1j/2)*np.kron(X,Y))
listOfm_2.append((1j/2)*np.kron(X,Z))
listOfm_2.append((1j/2)*np.kron(Y,Z))
listOfm_2.append((1j/2)*np.kron(Y,X))
listOfm_2.append((1j/2)*np.kron(Z,X))
listOfm_2.append((1j/2)*np.kron(Z,Y))

listOfk_2 = []
listOfk_2.append((1j/2)*np.kron(X,I))
listOfk_2.append((1j/2)*np.kron(Y,I))
listOfk_2.append((1j/2)*np.kron(Z,I))
listOfk_2.append((1j/2)*np.kron(I,X))
listOfk_2.append((1j/2)*np.kron(I,Y))
listOfk_2.append((1j/2)*np.kron(I,Z))

listOfh_2=[]
listOfh_2.append((1j/2)*np.kron(X,X))
listOfh_2.append((1j/2)*np.kron(Y,Y))
listOfh_2.append((1j/2)*np.kron(Z,Z))

listOff_2=[]

#Function that creates the Khaneja-Glaser basis for SU(2^N)
def create_KG_basis(N):
  #it returns the su(2) basis if N=2
  if N==2:
    return(listOfk_2,[],[],listOfm_2,listOfh_2,[])
  else:
  #Else it defines the KG-basis as described in Figures 1 and 4
    subasis=listOfm_2+listOfk_2
    for k in range(3,N+1):
      k_N_basis=[(1j/2)*np.kron(np.eye(2**(k-1)),Z)]
      k_N1_basis=[]
      k_N0_basis=[]
      m_N_basis=[(1j/2)*np.kron(np.eye(2**(k-1)),X),(1j/2)*np.kron(np.eye(2**(k-1)),Y)]
      for i in subasis:
        m_N_basis.append(np.kron(i,X))
        m_N_basis.append(np.kron(i,Y))
        k_N1_basis.append(np.kron(i,Z))
        k_N0_basis.append(np.kron(i,I))
      for l in k_N1_basis:
        k_N_basis.append(l)
      for r in k_N0_basis:
        k_N_basis.append(r)
      subasis=k_N_basis+m_N_basis

  h_2 = []
  h_2.append(['x','x'])
  h_2.append(['y','y'])
  h_2.append(['z','z'])
  s_n=[]
  for i in h_2:
    s_n.append(i)
  for k in range(3,N+1):
    k_n=[]
    F_n=[]
    for i in s_n:
      k_n.append(i+['x'])
      F_n.append(i+['z'])
    k_n.append(['i']*(k-1)+['x'])
    for i in range(0,len(s_n)):
      s_n[i]=s_n[i]+['i']
    for i in k_n:
      s_n.append(i)
  #We sorte these basis alphabetically because of the implementation of Step 5.
  k_n.sort()
  F_n.sort()
  h_N_basis=[]
  f_N_basis=[]
  for i in range(0,len(k_n)):
    for l in range(0,len(k_n[i])):
      k_n[i][l]=symbol_to_matrix[k_n[i][l]]

  for i in k_n:
    h_N_basis.append((1j/2)*ft.reduce(np.kron, i))

  for i in range(0,len(F_n)):
    for l in range(0,len(F_n[i])):
      F_n[i][l]=symbol_to_matrix[F_n[i][l]]


  for i in F_n:
    f_N_basis.append((1j/2)*ft.reduce(np.kron, i))

  return(k_N_basis,k_N1_basis,k_N0_basis,m_N_basis,h_N_basis,f_N_basis)

# These are a couple of general functions we will use throughout the code

#Inner product of two matrices X and Y
def innerP(X,Y):
  return(np.trace(X@(Y.conj().T)))


#commutator of X and Y
def com(X,Y):
  return(X@Y-Y@X)

#Project a matrix X into the subspace of matrix Y
def projXtoY(X,Y):
  return((innerP(X,Y)/innerP(Y,Y))*Y)


#projects the matrix X on a subspace generated by matrices in basis_list
def project_on_subspace(X,basis_list):
  X_proj=sum([projXtoY(X,i) for i in basis_list])
  return(X_proj)

#get the coordinates of matrix X on the normalized basis defined in basis_list
def write_on_basis(X,basis_list):
  res=[]
  for B in basis_list:
    res.append((innerP(X,B)).real)
  return np.array(res)


#Calculate the subspace error of matrix k in relation to the space generated by basis_list
def test_if_on_Torus(X,basis_list):
  err=[com(X,B) for B in basis_list]
  return np.linalg.norm(err)

#Function that calculates the distance of an element to a sucpace for debugging purpouses
def test_if_on_Subspace(k,basis):
  return(np.linalg.norm(k-project_on_subspace(k,basis)))


#Functions related to manipulating the phase from step (12), represented by the hat (^)

#This function removes the z element (adds, the hat)
def separate_z_factor(k):
  N=int(math.log(len(k),2))
  z_element=1j/2*Z
  for i in range(N-1):
    z_element=np.kron(I,z_element)
  #phase is self adjoint so we supress *
  z_factor=projXtoY(k,z_element)
  return(k-z_factor,z_factor)


def separate_Id_phase(g):
  Id=np.eye(len(g))
  Id_phase=innerP(g,Id)/innerP(Id,Id)
  return(g-Id_phase*Id,Id_phase)

This block is where the decomposition steps are implemented

In [ ]:
# implements the KHK decomposiyion, steps (3-9) of the step by step sumary
def KHK_decomposition(g,k_N_basis,m_N_basis,h_N_basis, display_error=False,optimization_retries=1):
  G=expm(g)
  Id=np.eye(len(g))

  #Create a normalized version of the k_N_basis
  k_N_basis_normalized=np.array([k/(innerP(k,k)**0.5) for k in k_N_basis])

  #Step (3): we are looking for an element on the span(k_basis), such that, when we calculate Ge^(−k), it falls on span(k_basis)
  #We define the function to be used on the optimizaton
  def T_of_k_optimization_function(X):
    #It first calculates the k elements from the coordinates
    k_test=np.sum(X[:, np.newaxis, np.newaxis] * k_N_basis_normalized, axis=0)
    #Now we calculate the respective m by Ge^(−k)
    m_test=logm(expm(-1*k_test)@G)
    #and  get the coordinates of its projection into the k_subspace
    m_proj=write_on_basis(m_test,k_N_basis_normalized)
    return m_proj

  #We use as the starting point of the optimization the projection of g on the k subspace
  X0 =write_on_basis(g,k_N_basis_normalized)

  # Find k0 coordinates using standard optimize.minimize
  sol_k =sc.optimize.root(T_of_k_optimization_function,X0)

  #Here the code retries the optimization with a random starting point, if the results were not succesfull, as many times as defined by optimization_retries
  if np.linalg.norm(sol_k.fun)>0.001:
    iterator = itertools.count() if optimization_retries == -1 else range(optimization_retries)
    for _ in iterator:
      print("Optimization error. Trying again.")
      X0=[random.random() for _ in range(len(k_N_basis))]
      sol_k =sc.optimize.root(T_of_k_optimization_function,X0)
      if np.linalg.norm(sol_k.fun)<0.001:
        break
    if np.linalg.norm(sol_k.fun)>0.001:
        raise AssertionError("Optimization failed")

  #Calculate k_0 from the coordinates
  k_0=sum([sol_i*k_i for sol_i,k_i in zip(sol_k.x,k_N_basis_normalized)])
  K_0=expm(k_0)


  #Step (4): calculate m
  #We also remove and store any potential id and z factors
  m, id_phase =separate_Id_phase(logm(expm(-1*k_0)@G))
  m, z_factor=separate_z_factor(m)


  #Step (5): define v=\sum{\pi^{j-1}*(u)_j}
  v=sum([math.pi**(i)*h_N_basis[i] for i in range(len(h_N_basis))])

  #Step (6): define the f_(m,v)(K) function
  def F_mv_of_k_optimization_function(K):
    #Calculate k_test from the coordinates
    k_test=np.sum(K[:, np.newaxis, np.newaxis] * k_N_basis_normalized, axis=0)
    #Get K_test through the exponential
    K_test=expm(k_test)
    #calculate the function
    F_mv_of_k_res=np.trace(np.dot(v,np.dot(K_test.conj().T,np.dot(m,K_test))))
    return (F_mv_of_k_res.real)

  #Step (7): find the coordinates of k1 by minimizing F_mv_of_k_optimization_function
  #using zero as the initial guess
  X0 = np.zeros(len(k_N_basis))
  sol_k = sc.optimize.minimize(F_mv_of_k_optimization_function,X0)

  #calculate k_1 from the coordinates
  k_1=sum([sol_i*k_i for sol_i,k_i in zip(sol_k.x,k_N_basis_normalized)])
  K_1=expm(k_1)

  #Step (8): calculate H and h
  h=(K_1.conj().T).dot(m.dot(K_1))
  H=expm(h)



  #If display_error==true, we show the errors corresponding to this KHK decomp
  if display_error:
    errors=pd.Series({'|G-K₀M|':np.linalg.norm(expm(g)-(K_0@expm(m)@(np.exp(id_phase)*Id))),
              '|m⟂|':test_if_on_Subspace(m,m_N_basis),
            '|g⟂|':test_if_on_Subspace(g,m_N_basis+k_N_basis),
            '|k₀⟂|':test_if_on_Subspace(k_0,k_N_basis),
            '|k₁⟂|':test_if_on_Subspace(k_1,k_N_basis),
            '|h⟂|':test_if_on_Subspace(h,h_N_basis),
            '|G-K₀K₁HK₁*I|':np.linalg.norm(G-K_0@K_1@H@(K_1).conj().T@(np.exp(id_phase)*Id))
            })
    display(pd.DataFrame([errors]))


  return([k_0,k_1,h],id_phase,z_factor)




#This function implements Steps (2-14) of the decomposition
def KG_decomposition(g_0, display_error=False,optimization_retries=1):
  #Calculate N of su(2^N)
  N=int(math.log(len(g_0),2))

  # Step (2) Compute the Khaneja-Glaser basis
  k_N_basis,k_N1_basis,k_N0_basis,m_N_basis,h_N_basis,f_N_basis=create_KG_basis(N)

  #This funcction implements Steps (3-9), it calculates the first KHK decomposition, and stores the identity phase
  G_decomp_0,id_phase,_=KHK_decomposition(g_0,k_N_basis,m_N_basis,h_N_basis,display_error,optimization_retries)
  G_decomp_0=dict(zip(["k_00","k_01","h"],G_decomp_0))

  #Step (10): prepare elements for next decomposition
  G_decomp_0["k_00@k_01"]=logm(expm(G_decomp_0["k_00"])@expm(G_decomp_0["k_01"]))
  G_decomp_0["k_01*"]=G_decomp_0["k_01"].conj().T
  #Separate the identity phase that might come from the log
  G_decomp_0["k_00@k_01"],new_id_phase=separate_Id_phase(G_decomp_0["k_00@k_01"])
  id_phase+=new_id_phase
  #remove the z factor (put on the hat)
  G_decomp_0["k_00@k_01-hat"],G_decomp_0["k_00@k_01-tilde"]=separate_z_factor(G_decomp_0["k_00@k_01"])
  G_decomp_0["k_01*-hat"],G_decomp_0["k_01*-tilde"]=separate_z_factor(G_decomp_0["k_01*"])

  #Steps (11) Calculate the secondary decompositions
  G_decomp_1,new_id_phase_10,new_z_factor_10=KHK_decomposition(G_decomp_0["k_00@k_01-hat"],k_N0_basis,k_N1_basis,f_N_basis,display_error,optimization_retries)
  G_decomp_2,new_id_phase_11,new_z_factor_11=KHK_decomposition(G_decomp_0["k_01*-hat"],k_N0_basis,k_N1_basis,f_N_basis,display_error,optimization_retries)

  #aggregate id phases
  id_phase+=new_id_phase_10
  id_phase+=new_id_phase_11

  #Agregate the results in a dictionaries
  G_decomp_1=dict(zip(["k_10","k_11","f_0"],G_decomp_1))
  G_decomp_2=dict(zip(["k_20","k_21","f_1"],G_decomp_2))

  #Step (12): Decompose the k fators into su(2^{n-1}) elements by removin identity tensor
  G_decomp_1['k_10´']=G_decomp_1['k_10'][::2,::2]
  G_decomp_1['k_11´']=G_decomp_1['k_11'][::2,::2]
  G_decomp_2['k_20´']=G_decomp_2['k_20'][::2,::2]
  G_decomp_2['k_21´']=G_decomp_2['k_21'][::2,::2]

  #Step (14): create a dictionary to store the final results.
  #We will store the abelian factors and z-factors as group elements and the ones that will be further decomposed as algebra elements
  KG_result={}

  #take the exponential of h and store it
  KG_result['H^0']=expm(G_decomp_0['h'])

  #aggregate the two z phases, remove the identity tensor products to get an SU2 element, take the exponential and store K^0-tilde and K^1-tilde.
  KG_result["K^0-tilde"]=expm((G_decomp_0["k_00@k_01-tilde"]+new_z_factor_10)[:2,:2])
  KG_result["K^1-tilde"]=expm((G_decomp_0["k_01*-tilde"]+new_z_factor_11)[:2,:2])

  #join neighboring k elements into a single k element
  KG_result['k_1']=logm(expm(G_decomp_1['k_10´'])@expm(G_decomp_1['k_11´']))
  KG_result['k_1'],new_id_phase=separate_Id_phase(KG_result['k_1'])
  id_phase+=new_id_phase

  KG_result['k_2']=G_decomp_1['k_11´'].conj().T

  KG_result['k_3']=logm(expm(G_decomp_2['k_20´'])@expm(G_decomp_2['k_21´']))
  KG_result['k_3'],new_id_phase=separate_Id_phase(KG_result['k_3'])
  id_phase+=new_id_phase

  KG_result['k_4']=G_decomp_2['k_21´'].conj().T

  #Take the exponenetial of the f elements
  KG_result["F^0"]=expm(G_decomp_1["f_0"])
  KG_result["F^1"]=expm(G_decomp_2["f_1"])

  #Step (13): Store the aggregated id_phase
  KG_result["id_phase"]=id_phase

  #Calculate the subspace error associated with the abelian elements
  subs_error={}
  subs_error['H^0']=test_if_on_Torus(G_decomp_0['h'],h_N_basis)
  subs_error['F^0']=test_if_on_Torus(G_decomp_1["f_0"],f_N_basis)
  subs_error['F^1']=test_if_on_Torus(G_decomp_2["f_1"],f_N_basis)

  return KG_result,subs_error


#This function implements Steps: (16-18) of the decomposition, factoring the SU(4) elements
def KG_decomposition_SU4(g_0):

  # calculate the su(4) kG-basis
  k_N_basis,k_N1_basis,k_N0_basis,m_N_basis,h_N_basis,f_N_basis=create_KG_basis(2)

  #Step (16): Calculate the KHK decomposition
  G_decomp,id_phase,_=KHK_decomposition(g_0,k_N_basis,m_N_basis,h_N_basis)
  #convert the result into a dictionary
  G_decomp=dict(zip(["k_0","k_1","h"],G_decomp))

  #Step (17): separate the left and right k-factors
  G_decomp["k_0L"]=project_on_subspace(G_decomp["k_0"],k_N_basis[:3])
  G_decomp["k_0R"]=project_on_subspace(G_decomp["k_0"],k_N_basis[3:])
  G_decomp["k_1L"]=project_on_subspace(G_decomp["k_1"],k_N_basis[:3])
  G_decomp["k_1R"]=project_on_subspace(G_decomp["k_1"],k_N_basis[3:])

  #Step (18): Store the decomposition elements. In this final decomp, everthing is stored as a group factor
  KG_result={}
  KG_result['id_phase']=id_phase

  KG_result['K^0']=expm(G_decomp['k_0L'][::2,::2])@expm(G_decomp['k_1L'][::2,::2])
  KG_result['K^1']=expm(G_decomp['k_0R'][:2,:2])@expm(G_decomp['k_1R'][:2,:2])
  KG_result['K^2']=expm(G_decomp['k_1L'][::2,::2]).conj().T
  KG_result['K^3']=expm(G_decomp['k_1R'][:2,:2]).conj().T

  KG_result["H^0"]=expm(G_decomp["h"])

  #Calculate the subspace error of H
  subs_error={}
  subs_error['H^0']=test_if_on_Torus(G_decomp['h'],h_N_basis)

  return KG_result,subs_error





This block contains the classes that store the decomposition factors

In [ ]:
#This class stores the elements of the KS-decomposition while it is still being computed
class storage_factors:
  def __init__(self,g,Id_phase):
    #These are the factors
    self.decomp_factors=[{"matrix": g,"type":'k',"N_Id":np.array([0,0])}]
    #This is the identity phase
    self.id_phase=Id_phase
    #This is the number of qubits su(2^N)
    self.N=int(math.log(len(g),2))
    #This is the initial matrix we are decomposing
    self.initial_g=g+self.id_phase*np.eye(2**self.N)

  #This function inserts into the decomp_factors list the new factors from subdecompositions
  def sub_decomp(self,index,KG_result,subs_error):
    self.id_phase=self.id_phase+KG_result['id_phase']

    #The N_Id list stores how many identity matrices are on the left and right of the factor
    N_Id=self.decomp_factors[index]["N_Id"]
    #The K_dim gives the N of the su(2^N) of the K factors
    K_dim=int(math.log(len(self.decomp_factors[index]["matrix"]),2))

    #Each facor is stored in a dictonary
    #It stores the matrix, the subspace error, the type and the number of identity entries in each side
    #the type explain if is is in one of the abelian groups (F or H), an SU(2) z-phase (Z) of a k-algebra element (k)
    new_factors=[]
    new_factors.append({"matrix": KG_result['k_1'],"error":0,"type":"k","N_Id":N_Id+[0,1]})
    new_factors.append({"matrix": KG_result['F^0'],"error":subs_error['F^0'],"type":"F","N_Id":N_Id})
    new_factors.append({"matrix": KG_result['k_2'],"error":0,"type":"k","N_Id":N_Id+[0,1]})
    new_factors.append({"matrix": KG_result['K^0-tilde'],"error":0,"type":"Z","N_Id":N_Id+[K_dim-1,0]})
    new_factors.append({"matrix": KG_result['H^0'],"error":subs_error['H^0'],"type":"H","N_Id":N_Id})
    new_factors.append({"matrix": KG_result['k_3'],"error":0,"type":"k","N_Id":N_Id+[0,1]})
    new_factors.append({"matrix": KG_result['F^1'],"error":subs_error['F^1'],"type":"F","N_Id":N_Id})
    new_factors.append({"matrix": KG_result['k_4'],"error":0,"type":"k","N_Id":N_Id+[0,1]})
    new_factors.append({"matrix": KG_result['K^1-tilde'],"error":0,"type":"Z","N_Id":N_Id+[K_dim-1,0]})

    self.decomp_factors=self.decomp_factors[:index]+new_factors+self.decomp_factors[index+1:]

  #This function inserts into the decomp_factors list the new factors from SU(4) decompositions
  def SU4_sub_decomp(self,index,KG_result,subs_error):
    self.id_phase=self.id_phase+KG_result['id_phase']

    N_Id=self.decomp_factors[index]["N_Id"]

    #Each facor is stored in a dictonary
    #It stores the matrix, the subspace error, the type and the number of identity entries in each side
    #the type explain if is is in the abelian group (H) or in the k-group (K)
    new_factors=[]
    new_factors.append({"matrix": KG_result['K^0'],"error":0,"type":"K","N_Id":N_Id+[0,1]})
    new_factors.append({"matrix": KG_result['K^1'],"error":0,"type":"K","N_Id":N_Id+[1,0]})
    new_factors.append({"matrix": KG_result['H^0'],"error":subs_error['H^0'],"type":"H","N_Id":N_Id})
    new_factors.append({"matrix": KG_result['K^2'],"error":0,"type":"K","N_Id":N_Id+[0,1]})
    new_factors.append({"matrix": KG_result['K^3'],"error":0,"type":"K","N_Id":N_Id+[1,0]})

    self.decomp_factors=self.decomp_factors[:index]+new_factors+self.decomp_factors[index+1:]

  #This function gets the indexes of all the k-algebra elements that are to be decomposed in the next iteration
  def recall_k_indexes(self):
    index_list=[]
    for index in range(len(self.decomp_factors)):
      if self.decomp_factors[index]["type"]=='k':
        index_list.append(index)
    return index_list

  #This function retrieves the matrix given an index
  def get_matrix(self,index):
    return(self.decomp_factors[index]['matrix'])


#This class is returned to the user after the decomposition is completed
class display_decomp_factors:
  def __init__(self, storage_factors):
    #uses the first id phase to get the total size
    self.N=storage_factors.N
    self.id_phase=np.exp(storage_factors.id_phase)
    self.decomp_elements=storage_factors.decomp_factors
    self.initial_G=expm(storage_factors.initial_g)
    #We now add indexes to the name of each type of factor
    indexes={'K':0,"F":0,"H":0,'Z': 0}
    for factor in self.decomp_elements:
      factor["name"]=factor["type"]+'_{'+str(indexes[factor["type"]])+"}"
      indexes[factor["type"]]+=1

  #This function multiplies all the factors to approximate the initial matrix G
  def approximate_G(self):
    G_approx=self.id_phase*np.eye(2**self.N)
    for factor in self.decomp_elements:
      #make a copy of the matrix and tensor with the necessary identities
      factorxI=factor['matrix'][:]
      for i in range(factor["N_Id"][0]):
        factorxI=np.kron(I,factorxI)
      for i in range(factor["N_Id"][1]):
        factorxI=np.kron(factorxI,I)
      G_approx=G_approx@factorxI
    return(G_approx)

  #This function returns the latex code for a profile of the decomposition, with the names and positions of each factor
  def print_decomp(self):
    decomp_str="Id^{i\phi}\cdot "
    for factor in self.decomp_elements:
      new_str=factor["name"]
      if factor["N_Id"][0]:
        new_str=r'I^{\otimes '+str(factor['N_Id'][0])+r'}\otimes '+new_str
      if factor["N_Id"][1]:
        new_str=new_str+r'\otimes I^{\otimes '+str(factor['N_Id'][1])+'}'
      decomp_str=decomp_str+new_str+r"\cdot "
    decomp_str=decomp_str[:-6]
    display(Math(decomp_str))
    return(decomp_str)

  #This function returns the matrix given its name
  def get_matrix_by_name(self,name):
    for factor in self.decomp_elements:
      if factor['name']==name:
        return factor['matrix']
    raise Exception('The name does not correspont to an element of the decomposition.')

  #This is a helper function that retrieves the list indexes of all abelian factors
  def get_index_abelian(self):
    list_indexes=[]
    for index in range(len(self.decomp_elements)):
      if self.decomp_elements[index]['type']=="F" or self.decomp_elements[index]['type']=="H" :
        list_indexes.append(index)
    return(list_indexes)

  #This function displays the erros (approximation and subspace) associated with the decomposition
  def get_errors(self):
    error_dict={'|G-Ĝ|':np.linalg.norm(self.initial_G-self.approximate_G())}
    error_dict.update({self.decomp_elements[index]['name']:self.decomp_elements[index]['error'] for index in self.get_index_abelian()})
    error_series=pd.Series(error_dict)
    #error_series = error_series.apply(lambda x: '{:0.2e}'.format(x))
    error_df= pd.DataFrame([error_series])
    #display(error_df)
    return(error_df)


#Required to display latex code
from IPython.display import HTML, Math
display(HTML("<script src='https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/"
"latest.js?config=default'></script>"))

This block is where the decomposition in iterated

In [ ]:
#This function iterates the KG-decomposition untill we are left only with SU(2) and abelian factors
#The parameter display_error is used for debugging, it shows the error at different points of the decomposition to see where the problem originated
#The parameter optimization_retries tells the code how many times it should retry to find k_0 and m in step 3. if optimization_retries=-1, it will retry unit it is successful
def iterated_KG_decomp(G, display_error=False, optimization_retries=1):
  #Step (1): calculate the log of g and store any identity phase produced
  g_0,id_phase=separate_Id_phase(logm(G))

  #Create the class to store the factors during the decomposition
  decomp_result=storage_factors(g_0,id_phase)

  #Calculate the KG-decomposition for each N
  N=int(math.log(len(G),2))
  for i_decomp in range(N-2):
    decomp_output_list={}
    #First the code gets the index of every element the will be decomposed in the next iteration
    for K_index in decomp_result.recall_k_indexes():
      #This function calculates the KG-decomposition (Steps (2-18)) and stores the factors produced
      #This for loop can be computed in paralel
      decomp_output_list[K_index]=KG_decomposition(decomp_result.get_matrix(K_index),display_error,optimization_retries)

    #In the end, it imputs the results from all decompositions in the class
    #The shif_correction is necessary to account for the new elements that were introuced
    shift_correction=0
    for key_index in decomp_output_list.keys():
      decomp_result.sub_decomp(key_index+shift_correction*8,*decomp_output_list[key_index])
      shift_correction+=1

  #The same thing is done for the SU(4) ellements, with the respective function
  su4_decomp_outputs={}
  for K_index in decomp_result.recall_k_indexes():
    su4_decomp_outputs[K_index]=KG_decomposition_SU4(decomp_result.get_matrix(K_index))

  shift_correction=0
  for key_index in su4_decomp_outputs.keys():
    decomp_result.SU4_sub_decomp(key_index+shift_correction*4,*su4_decomp_outputs[key_index])
    shift_correction+=1

  return display_decomp_factors(decomp_result)


#TESTING THE CODE

In [ ]:
#Test of the code running with random matrices from SU(2^N)
from scipy.stats import unitary_group
import time

#Parameters of the test
number_matrices=30
number_qubits=3

#Here we will store the matrices generated and the results
results=[]
#This df will store the errors of all decompositions
error_df=pd.DataFrame()

start_time = time.time()
for t_par in range(number_matrices):
  U = unitary_group.rvs(2**number_qubits)
  G= U/np.linalg.det(U)**(1/(2**number_qubits))
  #Calculates the decomposition
  if t_par% 10 == 0:
        print("Iteration:", t_par)
  try:
    G_dcp=iterated_KG_decomp(G,display_error=False,optimization_retries=-1)
    error_df = pd.concat([error_df,G_dcp.get_errors()])
    results.append(G_dcp)
  except AssertionError as er:
    print("An error occurred:", er)
    pass


end_time = time.time()
execution_time = end_time - start_time

print(f"Each decomposition took {execution_time/number_matrices:.1f} seconds to compute.")

print("Percentage of succesfull decomps:", str(int(len(results)/number_matrices*100))+"%")


error_df=error_df.reset_index(drop=True)
pd.options.display.float_format = '{:.1e}'.format

display(error_df)


Iteration: 0
Iteration: 10
Iteration: 20
Each decomposition took 4.3 seconds to compute.
Percentage of succesfull decomps: 100%


,|G-Ĝ|,H_{0},F_{0},H_{1},H_{2},H_{3},F_{1},H_{4}
0,9.2e-11,5.2e-07,2.8e-06,3.5e-07,1.7e-06,4.6e-07,3.9e-06,4.6e-07
1,1.9e-09,8.3e-07,3.1e-06,7.6e-07,3.0e-06,1.8e-07,1.6e-06,1.1e-06
2,3.3e-10,4.6e-07,1.3e-06,2.4e-06,2.3e-06,9.7e-07,9.9e-07,2.8e-06
3,2.2e-10,2.8e-07,4.0e-06,2.4e-06,4.4e-06,1.4e-06,1.7e-06,4.2e-07
4,2.3e-10,5.0e-07,1.3e-06,1.6e-06,2.4e-06,2.5e-06,1.8e-06,8.2e-07
5,3.6e-10,1.3e-06,4.6e-06,9.3e-07,2.3e-06,1.8e-06,1.3e-06,5.2e-07
6,2.4e-10,1.6e-06,4.9e-06,7.8e-07,2.3e-06,3.4e-06,7.1e-07,3.6e-07
7,1.6e-10,2.6e-07,1.0e-06,1.4e-06,2.3e-06,9.4e-07,1.2e-06,8.1e-07
8,2.0e-10,1.2e-06,3.9e-07,1.4e-07,1.0e-06,1.6e-06,2.0e-06,3.9e-07
9,4.2e-10,6.8e-07,3.7e-06,2.8e-06,2.2e-06,9.9e-07,7.0e-06,1.5e-06


# Tutorial on how to use the function

In [ ]:
#When imputing your matrix G
U = unitary_group.rvs(2**3)
G= U/np.linalg.det(U)**(1/(2**3))

#After calculating the decomposition
G_dcp=iterated_KG_decomp(G)

#We can visualize the result with
G_dcp.print_decomp()

<IPython.core.display.Math object>

'Id^{i\\phi}\\cdot K_{0}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{1}\\otimes I^{\\otimes 1}\\cdot H_{0}\\otimes I^{\\otimes 1}\\cdot K_{2}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{3}\\otimes I^{\\otimes 1}\\cdot F_{0}\\cdot K_{4}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{5}\\otimes I^{\\otimes 1}\\cdot H_{1}\\otimes I^{\\otimes 1}\\cdot K_{6}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{7}\\otimes I^{\\otimes 1}\\cdot I^{\\otimes 2}\\otimes Z_{0}\\cdot H_{2}\\cdot K_{8}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{9}\\otimes I^{\\otimes 1}\\cdot H_{3}\\otimes I^{\\otimes 1}\\cdot K_{10}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{11}\\otimes I^{\\otimes 1}\\cdot F_{1}\\cdot K_{12}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{13}\\otimes I^{\\otimes 1}\\cdot H_{4}\\otimes I^{\\otimes 1}\\cdot K_{14}\\otimes I^{\\otimes 2}\\cdot I^{\\otimes 1}\\otimes K_{15}\\otimes I^{\\otimes 1}\\cdot I^{\\otimes 2}\\otim

In [ ]:
#You can get one specific matrix by
G_dcp.get_matrix_by_name("K_{0}")
#Don't forget the curly brackets!

array([[-0.64481123-0.0512798j ,  0.75685798+0.09356734j],
       [-0.75685798+0.09356734j, -0.64481123+0.0512798j ]])

In [ ]:
#And also print the errors of the decomposition
G_dcp.get_errors()

,|G-Ĝ|,H_{0},F_{0},H_{1},H_{2},H_{3},F_{1},H_{4}
0,3.5e-10,3.3e-06,4.7e-07,3.8e-07,3.1e-06,4.1e-07,1.4e-06,1.2e-06


In [ ]:
#You can also customize whether the function displays mid-decomp erros with the display_error flag
#And how many times it retries the Step (3) decomposition. optimization_retries=-1 retries unitil it is successfull
U = unitary_group.rvs(2**3)
G= U/np.linalg.det(U)**(1/(2**3))

G_dcp=iterated_KG_decomp(G,display_error=True,optimization_retries=2)

,|G-K₀M|,|m⟂|,|g⟂|,|k₀⟂|,|k₁⟂|,|h⟂|,|G-K₀K₁HK₁*I|
0,5.7e-11,7.8e-10,7.7e-16,3.9e-16,5.2e-16,3.7e-06,5.7e-11


,|G-K₀M|,|m⟂|,|g⟂|,|k₀⟂|,|k₁⟂|,|h⟂|,|G-K₀K₁HK₁*I|
0,3.6e-15,5.9e-10,1.4e-15,3.2e-16,2.5e-16,8.7e-07,3.7e-15


,|G-K₀M|,|m⟂|,|g⟂|,|k₀⟂|,|k₁⟂|,|h⟂|,|G-K₀K₁HK₁*I|
0,3.0e-15,2.3e-09,5.3e-16,2.3e-16,1.9e-16,1.1e-06,3.2e-15


NOW EXPLAIN HOW TO use THE FUNCTIONs OF THE DISPLAY FACTOR